In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16818716
paper_name = 'lam_conibear_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits_genes_data.txt', sep='\t', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 8 x 2


In [10]:
original_data.head()

,0,1
0,chs4,0.29
1,chs3,0.30
2,chs6,0.30
3,chs5,0.39
4,slt2,0.40


In [11]:
original_data['gene'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, gene, orf]
Index: []


In [15]:
original_data['data'] = pd.to_numeric(original_data[1], errors='coerce')

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(8, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [138]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,138
data_type,value
orf,
YBL061C,0.29
YBR023C,0.30
YHR030C,0.40
YHR142W,0.52
YJL095W,0.46


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,138
,data_type,value
gene_id,orf,
149,YBL061C,0.29
219,YBR023C,0.30
997,YHR030C,0.40
1109,YHR142W,0.52
3338,YJL095W,0.46


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        138           
data_type       value     valuez
gene_id orf                     
149     YBL061C  0.29  16.221851
219     YBR023C  0.30  16.781225
997     YHR030C  0.40  22.374967
1109    YHR142W  0.52  29.087457
3338    YJL095W  0.46  25.731212

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16818716...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00, 12.71it/s]

Updating the data_modified_on field...
